In [1]:
import os

# TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL=1
os.environ["TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL"] = "1"

In [2]:
from torch.nn.functional import scaled_dot_product_attention as sdpa
import torch
from torch.nested import nested_tensor, as_nested_tensor, to_padded_tensor
from torch.nn.attention import sdpa_kernel, SDPBackend
import einops

In [3]:
sequence_lengths = [5,3,4]
batch_size = len(sequence_lengths)
dim = 16
num_heads = 2

In [4]:
index = 0
doc_ids = []
for s in sequence_lengths:
    doc_ids.extend([index] * s)
    index += 1

In [5]:
doc_ids

[0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2]

In [6]:
torch.tensor(doc_ids).unsqueeze(0).flatten()

tensor([0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2])

In [7]:
sq = torch.bincount(torch.tensor(doc_ids))

In [8]:
xs = []
for length in sequence_lengths:
    x = torch.randn(length, dim, device="cuda")
    xs.append(x)
flat = torch.cat(xs, dim=0)

In [9]:
flat

tensor([[ 1.4916,  1.7849,  3.4046,  1.7151, -1.1746,  0.2285,  0.9106,  1.0361,
          0.5585, -0.9251, -0.4517, -1.0741, -0.0240, -0.5906,  1.7991,  2.0408],
        [-1.7310,  0.0368,  0.3829, -1.3196,  0.3982,  1.9520, -0.6129,  0.7469,
         -0.0440, -2.7454,  0.2252,  2.2554,  0.1222, -0.9095, -2.2464,  0.0089],
        [ 0.5272, -0.6069, -1.8869,  0.6377, -0.2734, -2.0784,  0.7324,  1.4362,
          0.3286, -0.5532, -0.9765,  0.4673, -0.3709,  0.8017, -0.8159, -0.3586],
        [-0.5215,  1.1200,  0.3122, -2.4374, -0.4252, -1.0416,  1.4994,  0.7259,
          0.9181, -0.2761,  0.3036, -1.1188, -1.4290,  0.7092,  1.6097, -0.2814],
        [-2.7919,  0.0480, -2.2163, -0.8090, -0.2155, -0.6701,  1.2593, -0.3077,
         -0.3419,  0.4163,  0.6302,  0.7038, -1.2802, -0.1003, -1.3733, -0.0531],
        [ 1.7614,  0.5977, -0.6840, -2.2159,  0.3322,  0.4414, -0.2041,  0.9757,
          0.4653,  0.8742,  0.2008,  0.0911,  0.3169, -0.6356, -0.7807, -1.3535],
        [-0.1889, -1.1

In [10]:
flat.shape

torch.Size([12, 16])

In [11]:
kproj = torch.nn.Linear(dim, dim, bias=False).to("cuda")

In [12]:
kproj(flat)

tensor([[-0.3179, -0.5119, -0.3939,  0.3105, -0.3195, -1.0871, -1.2588, -0.5592,
          0.0608,  0.8394, -0.5161, -0.6072,  0.2392, -0.7779,  0.5413,  0.1489],
        [ 0.1590, -0.5374,  0.7271,  1.0842,  1.0605,  0.2639, -0.7809,  1.7747,
         -1.2983, -0.0957,  0.0215,  0.6926,  0.5073,  1.6882,  1.1316,  0.6247],
        [ 0.0418, -0.1002,  0.2572,  0.7253,  1.2750,  0.5519, -0.0871, -0.8624,
          0.1862,  0.4015, -0.3591, -0.2710, -0.1068, -0.3259, -0.1497,  0.0320],
        [ 0.2756, -0.3368, -0.5439, -0.7740, -0.4000,  0.5323, -0.3956,  0.5878,
          0.8014,  0.2938, -0.2045, -0.0165, -0.0716,  0.0383,  0.3207, -0.2383],
        [ 0.9160, -0.1710,  0.3380, -0.1874,  0.5442,  0.3302,  0.7345,  0.2012,
         -0.6109,  0.1152,  0.3147,  0.3801,  0.1619,  0.5435, -0.7788, -0.5265],
        [-0.2497,  0.1535,  0.8521, -0.0743, -0.4607,  1.5720, -0.2323,  0.5835,
          0.4038, -0.1741,  0.2414,  0.8437, -0.1266, -0.3188,  0.4302,  0.2088],
        [ 0.7655,  0.7

In [13]:
re = einops.rearrange(flat, '(batch seq) (heads headdim) -> (batch seq) heads headdim', heads=num_heads, batch=batch_size)

In [14]:
len(sq)

3

In [15]:
splits = re.split(sq.tolist(), dim=0)

In [16]:
sequence_lengths

[5, 3, 4]

In [17]:
type(splits)

tuple

In [18]:
len(splits)

3

In [19]:
nest = nested_tensor(list(splits))

/media/john/Tertiary/Projects/ML/BayesianFlowNet/.venv/lib/python3.12/site-packages/torch/nested/__init__.py:250: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(


In [20]:
nest

nested_tensor([
  tensor([[[ 1.4916,  1.7849,  3.4046,  1.7151, -1.1746,  0.2285,  0.9106,
             1.0361],
           [ 0.5585, -0.9251, -0.4517, -1.0741, -0.0240, -0.5906,  1.7991,
             2.0408]],
  
          [[-1.7310,  0.0368,  0.3829, -1.3196,  0.3982,  1.9520, -0.6129,
             0.7469],
           [-0.0440, -2.7454,  0.2252,  2.2554,  0.1222, -0.9095, -2.2464,
             0.0089]],
  
          [[ 0.5272, -0.6069, -1.8869,  0.6377, -0.2734, -2.0784,  0.7324,
             1.4362],
           [ 0.3286, -0.5532, -0.9765,  0.4673, -0.3709,  0.8017, -0.8159,
            -0.3586]],
  
          [[-0.5215,  1.1200,  0.3122, -2.4374, -0.4252, -1.0416,  1.4994,
             0.7259],
           [ 0.9181, -0.2761,  0.3036, -1.1188, -1.4290,  0.7092,  1.6097,
            -0.2814]],
  
          [[-2.7919,  0.0480, -2.2163, -0.8090, -0.2155, -0.6701,  1.2593,
            -0.3077],
           [-0.3419,  0.4163,  0.6302,  0.7038, -1.2802, -0.1003, -1.3733,
            -0.0531]

In [21]:
#nest.transpose(1,2).shape

In [22]:
nest = nest.transpose(1,2) # batch x heads x seq x headdim

In [23]:
nest

nested_tensor([
  tensor([[[ 1.4916,  1.7849,  3.4046,  1.7151, -1.1746,  0.2285,  0.9106,
             1.0361],
           [-1.7310,  0.0368,  0.3829, -1.3196,  0.3982,  1.9520, -0.6129,
             0.7469],
           [ 0.5272, -0.6069, -1.8869,  0.6377, -0.2734, -2.0784,  0.7324,
             1.4362],
           [-0.5215,  1.1200,  0.3122, -2.4374, -0.4252, -1.0416,  1.4994,
             0.7259],
           [-2.7919,  0.0480, -2.2163, -0.8090, -0.2155, -0.6701,  1.2593,
            -0.3077]],
  
          [[ 0.5585, -0.9251, -0.4517, -1.0741, -0.0240, -0.5906,  1.7991,
             2.0408],
           [-0.0440, -2.7454,  0.2252,  2.2554,  0.1222, -0.9095, -2.2464,
             0.0089],
           [ 0.3286, -0.5532, -0.9765,  0.4673, -0.3709,  0.8017, -0.8159,
            -0.3586],
           [ 0.9181, -0.2761,  0.3036, -1.1188, -1.4290,  0.7092,  1.6097,
            -0.2814],
           [-0.3419,  0.4163,  0.6302,  0.7038, -1.2802, -0.1003, -1.3733,
            -0.0531]]], device='

In [24]:
torch.tensor([[[-2.2262e+00, -9.5008e-01,  2.7648e-01, -1.5062e-01, -2.0740e-01,
            -3.0419e-01, -1.4584e+00, -1.8132e+00],
           [ 3.0436e-01,  2.0710e+00,  1.0387e+00,  1.0599e+00, -8.9734e-01,
            -6.5138e-01, -6.2088e-01, -8.3862e-01],
           [ 1.4547e-01,  2.9574e-02, -2.8565e-01,  2.0579e+00,  2.0022e-01,
            -1.2525e-01, -8.1289e-01, -2.5735e-01],
           [-1.4110e+00, -4.9433e-01,  7.6466e-02,  5.9130e-01, -1.3501e+00,
            -6.3912e-01, -2.2483e+00, -8.1429e-01],
           [ 7.6943e-01, -1.3943e-01,  5.8620e-01,  5.7719e-01,  2.1797e+00,
             1.3655e+00,  2.9225e-01, -2.9104e-01]],
  
          [[ 3.7461e-01,  1.5293e+00, -5.9289e-01,  2.0810e+00, -1.9876e+00,
            -5.5002e-01, -8.8987e-01,  1.2942e+00],
           [ 4.7834e-01,  4.2236e-01,  8.3717e-04,  2.4919e-01, -4.9489e-01,
            -9.4589e-01, -1.4299e+00,  3.7448e-01],
           [-1.2941e+00,  2.5023e-01, -8.5503e-01, -2.2924e+00,  1.7045e-01,
            -4.7716e-02,  7.2786e-01, -1.4844e+00],
           [ 7.1210e-01,  3.4433e-01,  8.2291e-01,  1.5901e+00,  4.9451e-01,
             5.6877e-01,  3.2368e-01,  1.9941e-01],
           [ 4.9133e-01, -6.6065e-02, -1.1930e-01, -1.0414e+00,  9.9746e-01,
             6.4476e-01,  1.2649e-01,  2.8418e-01]]], device='cuda:0').shape

torch.Size([2, 5, 8])

In [25]:
fixed_s = 3
queries = torch.rand(batch_size, fixed_s, dim, device="cuda", requires_grad=True)

In [26]:
q = torch.randn(fixed_s, dim, device="cuda")

In [27]:
einops.rearrange(
            einops.repeat(q, "seq dim -> batch seq dim", batch=batch_size),
            "batch seq (heads headdim) -> batch seq heads headdim",
            heads=num_heads,
        )

tensor([[[[ 0.5009, -1.8686, -1.3037,  0.4697, -2.4740,  0.4507, -1.5182,
           -0.7960],
          [ 1.2471,  0.5087,  1.3876, -0.3216,  0.4658, -0.8834,  0.4997,
           -0.4570]],

         [[ 0.5142,  1.8211, -1.1335,  1.0116,  0.6607,  0.1001, -0.3147,
           -2.2215],
          [ 1.8350,  1.0325, -0.3487, -0.3852, -0.1719,  0.5856, -0.2714,
            1.3793]],

         [[-0.0557, -0.1217,  1.0818, -0.5440,  0.3747,  0.2173, -0.8971,
           -0.6712],
          [-0.8019, -1.5829,  1.3642, -0.3016,  0.4880, -0.5979,  0.4647,
           -0.4773]]],


        [[[ 0.5009, -1.8686, -1.3037,  0.4697, -2.4740,  0.4507, -1.5182,
           -0.7960],
          [ 1.2471,  0.5087,  1.3876, -0.3216,  0.4658, -0.8834,  0.4997,
           -0.4570]],

         [[ 0.5142,  1.8211, -1.1335,  1.0116,  0.6607,  0.1001, -0.3147,
           -2.2215],
          [ 1.8350,  1.0325, -0.3487, -0.3852, -0.1719,  0.5856, -0.2714,
            1.3793]],

         [[-0.0557, -0.1217,  1.0818, 

In [28]:
queries = einops.rearrange(queries, 'batch seq (heads headdim) -> batch seq heads headdim', heads=num_heads)

In [29]:
nested_queries = as_nested_tensor(queries)
nested_queries = nested_queries.transpose(1,2) # batch x heads x seq x headdim

In [32]:
nested_queries = nested_queries.contiguous()
nest = nest.contiguous()

In [33]:
with sdpa_kernel(SDPBackend.MATH):
    output = sdpa(query=nested_queries, key=nest, value=nest)

In [34]:
o2 = to_padded_tensor(output.contiguous(), padding=0.0)

In [35]:
grad = torch.autograd.grad(o2.sum(), queries, retain_graph=True)[0]

In [36]:
queries = queries - grad

In [37]:
mock_target = torch.randn_like(queries)

In [38]:
(queries - mock_target).pow(2).mean().backward()

In [39]:
o2.shape

torch.Size([3, 2, 3, 8])

In [40]:
einops.rearrange(o2, 'batch heads seq headdim -> batch seq (heads headdim)').shape

torch.Size([3, 3, 16])

In [41]:
flat.shape

torch.Size([12, 16])

In [42]:
experimental = torch.rand(batch_size, fixed_s, dim)

In [43]:
qe = einops.rearrange(experimental, 'batch seq (heads headdim) -> batch seq heads headdim', heads=num_heads)

In [44]:
nested_qe = as_nested_tensor(list(qe.unbind(0)))

In [45]:
nested_qe

nested_tensor([
  tensor([[[0.9803, 0.4859, 0.8780, 0.2129, 0.5846, 0.8779, 0.1511, 0.6194],
           [0.6779, 0.4188, 0.0123, 0.2945, 0.7336, 0.5283, 0.0934, 0.3260]],
  
          [[0.5708, 0.0867, 0.8114, 0.5510, 0.5883, 0.6585, 0.0722, 0.1329],
           [0.0012, 0.0980, 0.3312, 0.6237, 0.9179, 0.3604, 0.6043, 0.7802]],
  
          [[0.6350, 0.2044, 0.9108, 0.1736, 0.6756, 0.9820, 0.9751, 0.1062],
           [0.9949, 0.7549, 0.7272, 0.3298, 0.4706, 0.6027, 0.2981, 0.5913]]]),
  tensor([[[0.2217, 0.3126, 0.7482, 0.1802, 0.4861, 0.3480, 0.0148, 0.6154],
           [0.1557, 0.8428, 0.1372, 0.7948, 0.9296, 0.3549, 0.0408, 0.6714]],
  
          [[0.3002, 0.8052, 0.2176, 0.4645, 0.7052, 0.9726, 0.7413, 0.2926],
           [0.4784, 0.3358, 0.3644, 0.1646, 0.1357, 0.1897, 0.0676, 0.1727]],
  
          [[0.2777, 0.9673, 0.1630, 0.7289, 0.8864, 0.5955, 0.4307, 0.7984],
           [0.1640, 0.8235, 0.0022, 0.3605, 0.0146, 0.4183, 0.2605, 0.9808]]]),
  tensor([[[0.1661, 0.2110, 0.8596, 0.

In [46]:
torch.tensor([[[0.5869, 0.3770, 0.3867, 0.1296, 0.2218, 0.6641, 0.2360, 0.0389],
           [0.6987, 0.2130, 0.9188, 0.2107, 0.3545, 0.1959, 0.0767, 0.8147]],
  
          [[0.2393, 0.6337, 0.7562, 0.6052, 0.6903, 0.5089, 0.0112, 0.5812],
           [0.9597, 0.7890, 0.5901, 0.5717, 0.4516, 0.2831, 0.7150, 0.8106]],
  
          [[0.5010, 0.4228, 0.1110, 0.2464, 0.6103, 0.6153, 0.8534, 0.4391],
           [0.3438, 0.3899, 0.6794, 0.1392, 0.4958, 0.0500, 0.9673, 0.9875]]]).shape

torch.Size([3, 2, 8])

In [47]:
nested_qe.transpose(1,2).contiguous()

nested_tensor([
  tensor([[[0.9803, 0.4859, 0.8780, 0.2129, 0.5846, 0.8779, 0.1511, 0.6194],
           [0.5708, 0.0867, 0.8114, 0.5510, 0.5883, 0.6585, 0.0722, 0.1329],
           [0.6350, 0.2044, 0.9108, 0.1736, 0.6756, 0.9820, 0.9751, 0.1062]],
  
          [[0.6779, 0.4188, 0.0123, 0.2945, 0.7336, 0.5283, 0.0934, 0.3260],
           [0.0012, 0.0980, 0.3312, 0.6237, 0.9179, 0.3604, 0.6043, 0.7802],
           [0.9949, 0.7549, 0.7272, 0.3298, 0.4706, 0.6027, 0.2981, 0.5913]]]),
  tensor([[[0.2217, 0.3126, 0.7482, 0.1802, 0.4861, 0.3480, 0.0148, 0.6154],
           [0.3002, 0.8052, 0.2176, 0.4645, 0.7052, 0.9726, 0.7413, 0.2926],
           [0.2777, 0.9673, 0.1630, 0.7289, 0.8864, 0.5955, 0.4307, 0.7984]],
  
          [[0.1557, 0.8428, 0.1372, 0.7948, 0.9296, 0.3549, 0.0408, 0.6714],
           [0.4784, 0.3358, 0.3644, 0.1646, 0.1357, 0.1897, 0.0676, 0.1727],
           [0.1640, 0.8235, 0.0022, 0.3605, 0.0146, 0.4183, 0.2605, 0.9808]]]),
  tensor([[[0.1661, 0.2110, 0.8596, 0.0341, 0.

In [48]:
torch.tensor([[[0.5869, 0.3770, 0.3867, 0.1296, 0.2218, 0.6641, 0.2360, 0.0389],
           [0.2393, 0.6337, 0.7562, 0.6052, 0.6903, 0.5089, 0.0112, 0.5812],
           [0.5010, 0.4228, 0.1110, 0.2464, 0.6103, 0.6153, 0.8534, 0.4391]],
  
          [[0.6987, 0.2130, 0.9188, 0.2107, 0.3545, 0.1959, 0.0767, 0.8147],
           [0.9597, 0.7890, 0.5901, 0.5717, 0.4516, 0.2831, 0.7150, 0.8106],
           [0.3438, 0.3899, 0.6794, 0.1392, 0.4958, 0.0500, 0.9673, 0.9875]]]).shape

torch.Size([2, 3, 8])

In [49]:
unbound = nest.transpose(1,2).unbind(0)

In [50]:
unbound[0].shape, unbound[1].shape

(torch.Size([5, 2, 8]), torch.Size([3, 2, 8]))

In [51]:
torch.cat(unbound, dim=0).shape

torch.Size([12, 2, 8])